In [1]:
# AOC Day 6

%load_ext autoreload
%autoreload 2

from aoc.utils import *

In [ ]:
width, height = 0, 0
def get_input():
    global width, height
    # Load input and test data
    with open("input.txt", "r") as file:
        input_data = list(map(lambda s: [c for c in s], file.read().strip().split("\n")))
    width, height = len(input_data[0]), len(input_data)

    guard_position = (0, 0)
    guard_momentum = (-1, 0)

    obstacles: set[tuple[int, int]] = set()
    for r, row in enumerate(input_data):
        for c, cell in enumerate(row):
            if cell == "^":
                guard_position = (r, c)
                continue
            if cell != "#":
                continue
            obstacles.add((r, c))
    return guard_position, guard_momentum, obstacles

def add_momentum(position: tuple[int, int], momentum: tuple[int, int]) -> tuple[int, int]:
    return (position[0] + momentum[0], position[1] + momentum[1])

def turn_right(momentum: tuple[int, int]) -> tuple[int, int]:
    return (momentum[1], -momentum[0])

def in_bounds(r: int, c: int) -> bool:
    global width, height
    return 0 <= r < height and 0 <= c < width

def will_path_loop(position: tuple[int, int], momentum: tuple[int, int], obstacles: set[tuple[int, int]], visited: set[tuple[tuple[int, int], tuple[int, int]]]) -> bool:
    while in_bounds(*position):
        if (position, momentum) in visited:
            return True
        visited.add((position, momentum))

        # Check if we will hit an obstacle
        if add_momentum(position, momentum) in obstacles:
            momentum = turn_right(momentum)
            continue
        position = add_momentum(position, momentum)
    return False
    
def run_path(starting_position: tuple[int, int], momentum: tuple[int, int], obstacles: set[tuple[int, int]]) -> int:
    position = starting_position
    loopable_positions: set[tuple[int, int]] = set()
    unique_positions: set[tuple[int, int]] = set()
    path: set[tuple[tuple[int, int], tuple[int, int]]] = set()
    while in_bounds(*position):
        unique_positions.add(position)
        # Do a normal obstacle check
        next_position = add_momentum(position, momentum)
        if next_position in obstacles:
            path.add((position, momentum))
            momentum = turn_right(momentum)
            continue

        # Check that if we were to add an obstacle, we would not loop
        if next_position not in unique_positions:
            copied_obstacles = obstacles.copy()
            copied_obstacles.add(next_position)
            if will_path_loop(position, momentum, copied_obstacles, path.copy()):
                loopable_positions.add(next_position)

        # Proceed as normal
        path.add((position, momentum))
        position = add_momentum(position, momentum)

    return len(unique_positions), len(loopable_positions)

part_1, part_2 = run_path(*get_input())
print(f"Part 1: {part_1}")
print(f"Part 2: {part_2}")